In [ ]:

# Install esm and other dependencies
! pip install esm
! pip install matplotlib




In [ ]:
#Set up Forge client for ESM C
from getpass import getpass
from esm.sdk import client

model = client(
    model="esmc-300m-2024-12", url="https://forge.evolutionaryscale.ai", token='MoBiPauli'
)

In [4]:

from concurrent.futures import ThreadPoolExecutor
from typing import Sequence

from esm.sdk.api import (
    ESM3InferenceClient,
    ESMProtein,
    ESMProteinError,
    LogitsConfig,
    LogitsOutput,
    ProteinType,
)

EMBEDDING_CONFIG = LogitsConfig(
    sequence=True, return_embeddings=True, return_hidden_states=True
)


def embed_sequence(model: ESM3InferenceClient, sequence: str) -> LogitsOutput:
    protein = ESMProtein(sequence=sequence)
    protein_tensor = model.encode(protein)
    output = model.logits(protein_tensor, EMBEDDING_CONFIG)
    return output


def batch_embed(
    model: ESM3InferenceClient, inputs: Sequence[ProteinType]
) -> Sequence[LogitsOutput]:
    """Forge supports auto-batching. So batch_embed() is as simple as running a collection
    of embed calls in parallel using asyncio.
    """
    with ThreadPoolExecutor() as executor:
        futures = [
            executor.submit(embed_sequence, model, protein) for protein in inputs
        ]
        results = []
        for future in futures:
            try:
                results.append(future.result())
            except Exception as e:
                results.append(ESMProteinError(500, str(e)))
    return results

In [16]:
import pandas as pd
import numpy as np
from esm.sdk.api import ESM3InferenceClient, ESMProtein
# Eigene Datei laden
df = pd.read_csv("/Users/paulinekipp/Documents/GitHub/group04-team04/data/ab_ag_annotated_with_CDR_strings.tsv", sep="\t")
df = df.dropna(subset=["CDR_L3_string"])

# Modell initialisieren
model = ESM3InferenceClient()

# ESMProtein-Objekte erstellen
proteins = [ESMProtein(sequence=s) for s in df["CDR_L3_string"]]

# Embeddings berechnen
results = batch_embed(model, proteins)

# Nur erfolgreiche Ergebnisse behalten
valid_results = [r for r in results if hasattr(r, "embeddings")]

# Dann Mittelwert berechnen
df["embedding_avg"] = [np.mean(r.embeddings, axis=0) for r in valid_results]

ValueError: Length of values (0) does not match length of index (5254)

In [ ]:
#Quelle für ESMC
@misc{esm2024cambrian,
  author = {{ESM Team}},
  title = {ESM Cambrian: Revealing the mysteries of proteins with unsupervised learning},
  year = {2024},
  publisher = {EvolutionaryScale Website},
  url = {https://evolutionaryscale.ai/blog/esm-cambrian},
  urldate = {2024-12-04}
}